## 任务2

In [1]:
from insert import deliver_database
import pydicom
import gdcm
from pynetdicom import AE, evt, AllStoragePresentationContexts, debug_logger
from pynetdicom.sop_class import *
from pydicom.uid import ImplicitVRLittleEndian, ExplicitVRLittleEndian
import os
import sqlite3

debug_logger()

def handle_store(event):
    """Handle a C-STORE request event."""
    ds = event.dataset
    ds.file_meta = event.file_meta
    print(ds.file_meta.TransferSyntaxUID)
    dcm_path = ".\\data\\" + str(ds.SOPInstanceUID) + ".dcm"
    ds.save_as(dcm_path, write_like_original=False)

    reader = gdcm.ImageReader()
    reader.SetFileName(dcm_path)
    if not reader.Read():
        print("error")
    ds_ori = pydicom.dcmread(dcm_path)
    print(ds_ori.file_meta.TransferSyntaxUID)

    dcm_path_lossless = ".\\data\\" + str(ds.SOPInstanceUID) + "_lossless.dcm"
    change = gdcm.ImageChangeTransferSyntax()
    change.SetInput(reader.GetImage())
    change.SetTransferSyntax(gdcm.TransferSyntax(gdcm.TransferSyntax.JPEG2000Lossless))
    if not change.Change():
        print("error2")
    writer = gdcm.ImageWriter()
    writer.SetFileName(dcm_path_lossless)
    writer.SetFile(reader.GetFile())
    writer.SetImage(change.GetOutput())
    if not writer.Write():
        print("write")
    ds_jepgloseless = pydicom.dcmread(dcm_path_lossless)
    print(ds_jepgloseless.file_meta.TransferSyntaxUID)
    dcm_abspath = os.path.abspath(dcm_path_lossless)
    deliver_database(dcm_abspath)
    print('DCM file insert successfully...')
    return 0x0000

calling_ae_white_list=["sjy_SCU","calling_AE"]
SCU_IP_white_list=["127.0.0.1"]
def handle_assoc(event):
    print("-------------SCU Assoc Param------------------")
    print(event.assoc.requestor.primitive.calling_ae_title)
    print(event.assoc.requestor.primitive.called_ae_title)
    print(event.assoc.requestor.address)
    print(event.assoc.requestor.port)

    print("-----------SCP Assoc Param----------------------")
    print(event.assoc.acceptor.ae_title)
    print(event.assoc.acceptor.address)
    print(event.assoc.acceptor.port)
    
    if event.assoc.requestor.primitive.calling_ae_title not in calling_ae_white_list:
        event.assoc.acse.send_reject(0x02, 0x01, 0x01)
        time.sleep(0.1)
    if event.assoc.requestor.address not in SCU_IP_white_list:
        event.assoc.acse.send_reject(0x02, 0x01, 0x01)
        time.sleep(0.1)
    
    return 0x0000

#handlers = [(evt.EVT_C_STORE, handle_store)]
handlers = [(evt.EVT_C_STORE, handle_store),
         (evt.EVT_REQUESTED, handle_assoc)]

ae = AE()
ae.add_supported_context("1.2.840.10008.5.1.4.1.1.77.1.1", ImplicitVRLittleEndian)
ae.add_supported_context("1.2.840.10008.5.1.4.1.1.2", ImplicitVRLittleEndian)  # CT Image lossless
#ae.add_supported_context("1.2.840.10008.5.1.4.1.1.2", "1.2.840.10008.1.2.4.91")  # CT Image lossy1.2.840.5.1.2
# ae.add_supported_context(SOPClass("1.2.840.10008.5.1.4.1.1.4"))  # MRI Image
ae.add_supported_context(SOPClass("1.2.840.10008.5.1.4.1.1.4"))  # MRI Image
#print(ae.supported_contexts[0])
ae.start_server(("127.0.0.1", 11112), evt_handlers=handlers)

D: Request Parameters:
D: ======================= INCOMING A-ASSOCIATE-RQ PDU ========================
D: Their Implementation Class UID:      1.2.826.0.1.3680043.9.3811.2.0.2
D: Their Implementation Version Name:   PYNETDICOM_202
D: Application Context Name:    1.2.840.10008.3.1.1.1
D: Calling Application Name:    sjy_SCU
D: Called Application Name:     ANY-SCP
D: Their Max PDU Receive Size:  16382
D: Presentation Context:
D:   Context ID:        1 (Proposed)
D:     Abstract Syntax: =VL Endoscopic Image Storage
D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntax:
D:       =Implicit VR Little Endian
D: Requested Extended Negotiation: None
D: Requested Common Extended Negotiation: None
D: Requested Asynchronous Operations Window Negotiation: None
D: Requested User Identity Negotiation: None
D: ========================== END A-ASSOCIATE-RQ PDU ==========================
I: Accepting Association
D: Accept Parameters:
D: ======================= OUTGOING A-ASSOCIATE-AC PDU

-------------SCU Assoc Param------------------
sjy_SCU
ANY-SCP
127.0.0.1
65145
-----------SCP Assoc Param----------------------
PYNETDICOM
127.0.0.1
11112
1.2.840.10008.1.2
1.2.840.10008.1.2
1.2.840.10008.1.2.4.90
DCM file insert successfully...


In [3]:
import pydicom
ds = pydicom.read_file('E:\\project4\\test19.dcm')
print(ds)

Dataset.file_meta -------------------------------
(0002, 0002) Media Storage SOP Class UID         UI: VL Endoscopic Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.8.498.58884710752811329474853545173710241440
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.3.4
-------------------------------------------------
(0008, 0016) SOP Class UID                       UI: VL Endoscopic Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.8.498.58884710752811329474853545173710241440
(0008, 0023) Content Date                        DA: '20230113'
(0008, 0033) Content Time                        TM: '172240.894419'
(0008, 0060) Modality                            CS: 'US'
(0010, 0010) Patient's Name                      PN: '7'
(0010, 0020) Patient ID                          LO: '1.22.55.44.20230113.11894'
(0010, 0030) Patient's Birth Date

In [16]:
import pydicom
ds = pydicom.read_file('E:\\project4\\CT-1.dcm')
print(ds)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 194
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.14519.5.2.1.7777.9002.230416241423687526820485795741
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-1.4.31'
-------------------------------------------------
(0008, 0000) Group Length                        UL: 422
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.14519.5.2.1.7777.9002.2304